In [105]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stopWords = set(word.strip().lower() for word in open("stop_words_en.txt"))

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        word = word.lower()
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if word in stopWords:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1
        print "%s\t%d" % (word, 1)

Overwriting mapper.py


In [ ]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

In [ ]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

In [25]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [106]:
%%bash

OUT_DIR="wordcount_result"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

0%however	1
0&\mathrm{if	1
0(8)320-1234	1
0)).(1	2
0,03	1
0,1,...,n	1
0,1,0	1
0,1,\dots,n	1
0,5	1
0,50	1


17/11/23 14:19:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 14:19:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 14:19:44 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/23 14:19:44 INFO mapreduce.JobSubmitter: number of splits:2
17/11/23 14:19:44 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1511436559166_0054
17/11/23 14:19:44 INFO impl.YarnClientImpl: Submitted application application_1511436559166_0054
17/11/23 14:19:44 INFO mapreduce.Job: The url to track the job: http://dded3576db44:8088/proxy/application_1511436559166_0054/
17/11/23 14:19:44 INFO mapreduce.Job: Running job: job_1511436559166_0054
17/11/23 14:19:49 INFO mapreduce.Job: Job job_1511436559166_0054 running in uber mode : false
17/11/23 14:19:49 INFO mapreduce.Job:  map 0% reduce 0%
17/11/23 14:20:05 INFO mapreduce.Job:  map 23% reduce 0%
17/11/23 14:20:11 INFO mapreduce.Job:  map 34% reduce 0%
17/11/23 14:20:17 INFO 